In [2]:
from datasets import load_dataset
dataset_tilde = load_dataset("tilde_model", lang1="de", lang2="is")

Downloading:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Using custom data configuration de-is-lang1=de,lang2=is


Downloading:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset tilde_model downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/tilde_model/de-is-lang1=de,lang2=is/0.0.0/cea5250cab9d7f20be33fd2b08330b2e17bf4de020ded2719dda9b288e99a205. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
print(type(dataset_tilde['train']))

<class 'datasets.arrow_dataset.Dataset'>


In [4]:
print(dataset_tilde)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 399681
    })
})


In [5]:
print(dataset_tilde['train'][3]['translation']['de'])

1.1 Zum gegenwärtigen Zeitpunkt wird hart um die Unterstützung der Öffentlichkeit für Islands Antrag auf EU-Mitgliedschaft gerungen, wie die öffentliche Anhörung im September 2010 in Island bestätigte.


In [7]:
!pwd

/mnt/c/Users/oddur/Háskólinn/MalTaekni/Lokaverk/raektin


In [9]:
dataset_tilde['train'].to_parquet('../data/processed_data/tilde_model.parquet')

90436388

In [10]:

dataset_tab = load_dataset('parquet', data_files='../data/processed_data/tilde_model.parquet')

Using custom data configuration default-d3d5525f74025ea3


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/parquet/default-d3d5525f74025ea3/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(dataset_tab)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 399681
    })
})


{'text': '284\tIch spreche kein Japanisch.\t399287\tÉg tala ekki japönsku.'}


{'text': '284\tIch spreche kein Japanisch.\t399287\tÉg tala ekki japönsku.'}
